StyleGAN2-ADA 모델은 텐서플로우 1 버전에서만 작동

In [1]:
%tensorflow_version 1.x     

TensorFlow 1.x selected.


In [5]:
!pip install numpy==1.19.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


현재 코랩에서 어떤 GPU를 할당받았는지 확인하는 코드.

In [ ]:
!nvidia-smi

Sun Jun  5 15:13:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

구글드라이브와 현재 코랩을 연동.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 깃허브에서 StyleGAN2-ADA를 코랩으로 불러와 다운로드 

In [4]:
# Download the code
%cd /content/
!git clone https://github.com/dvschultz/stylegan2-ada.git
!mkdir downloads
!mkdir datasets

/content
Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 364, done.
remote: Total 364 (delta 0), reused 0 (delta 0), pack-reused 364
Receiving objects: 100% (364/364), 56.17 MiB | 42.42 MiB/s, done.
Resolving deltas: 100% (197/197), done.


## Dataset

데이터셋은 압축하여 .zip형태로 구글드라이브에 저장 

In [ ]:
dataset_name = "celeb"   

코랩으로 데이터셋 압축파일을 불러와서 압축을 품

In [ ]:
import zipfile
path = "/content/drive/MyDrive/Colab Notebooks/datasets/"
dataset = dataset_name + ".zip"
local_path = "/content/"
file_name = path + dataset
with zipfile.ZipFile(file_name, 'r') as zip:
   #zip.printdir()
   print('Extracting all the files now...') 
   zip.extractall(local_path) 
   print('Done!')

Extracting all the files now...
Done!


불러온 이미지 데이터셋 jpg 파일들을 텐서플로에서 사용할 수 있는 데이터셋 포맷(tfrecords)으로 바꿔야 함

In [ ]:
%cd /content/

#update this to the path to your image folder
dataset_folder_name = 'drive/MyDrive/data' # name of zip file may be different from folder name it extracted to
dataset_path = "/content/" + dataset_folder_name


#you don't need to edit anything here
%cd /content/stylegan2-ada
!python dataset_tool.py create_from_images /content/datasets/{dataset_name} {dataset_path}

/content
/content/stylegan2-ada
Loading images from "/content/drive/MyDrive/data"
Creating dataset "/content/datasets/celeb"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 70 images.


## Training

학습할 때 조절가능한 다양한 파라미터들을 --help를 통해 확인 가능 

In [7]:
%cd /content/stylegan2-ada
!python train.py --help

/content/stylegan2-ada
usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with L

아래 셀을 실행하면 학습이 시작되고 학습이 진행되는 동안에는 이 노트북 브라우저 창을 열어두어야 함. 노트북 창을 닫거나 인터넷 연결 종료, 오랜시간동안 실행되고 있는 경우 등으로 연결이 끊기면 처음 셀부터 다시 실행해야 함. resume_form의 경로를 최근 pkl파일 경로로 바꿔주면 이어서 학습이 가능함 

학습이 진행되는 동안 구글 드라이브 폴더를 확인하면 중간 과정의 이미지 경과물과 모델을 확인 가능 


In [ ]:
# output directory 결과물 저장 경로 
output_dir = '/content/drive/MyDrive/Colab\ Notebooks/results/' + dataset_name + "/"

# config
config = "auto"

# gamma
gamma = 1

# 학습진행 중 몇번째마다 저장할것인지 
snapshot_count = 1

# 좌우반전
mirrored = True
# 상하 반전 
mirroredY = False

#metrics
metric_list = None

# 학습을 이어하기 위한 resum_from
resume_from = "/content/drive/MyDrive/Colab\ Notebooks/results/celeb/00005-celeb-mirror-auto1-gamma1-resumecustom/network-snapshot-000028.pkl"


# 위의 파라미터들을 이용한 학습시작 
!python train.py --outdir={output_dir} \
                 --cfg={config} \
                 --snap={snapshot_count} \
                 --data=/content/datasets/{dataset_name} \
                 --mirror={mirrored} --mirrory={mirroredY} \
                 --gamma={gamma} \
                 --metrics={metric_list} \
                 --resume={resume_from}

tcmalloc: large alloc 4294967296 bytes == 0x8550000 @  0x7fac0d61e001 0x7fac0a82254f 0x7fac0a872b58 0x7fac0a876b17 0x7fac0a915203 0x593835 0x548c51 0x5127f1 0x549e0e 0x4bca8a 0x532b86 0x594a96 0x548cc1 0x5127f1 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549e0e 0x4bca8a 0x5134a6 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2
tcmalloc: large alloc 4294967296 bytes == 0x7faa246cc000 @  0x7fac0d61c1e7 0x7fac0a82246e 0x7fac0a872c7b 0x7fac0a87335f 0x7fac0a915103 0x593835 0x548c51 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x549e0e
tcmalloc: large alloc 4294967296 bytes == 0x7fa9236ca000 @  0x7fac0d61c1e7 0x7fac0a82246e 0x7fac0a872c7b 0x7fac0a87335f 0x7fabb643c235 0x7fabb5dbf792 0x7fabb5dbfd42 0x7fabb5d78aee 0x59371f 0x548c51 0x51566f 0x593dd7 0x511e2c 0x549e0e 0x4bcb19 0x5134a6 0x549576 0x593f

In [ ]:
!python generate.py --outdir=/content/out/images/ --trunc=0.8 --seeds=0 --network=/content/drive/MyDrive/ladiescrop-network-snapshot-012885.pkl